<a href="https://colab.research.google.com/github/alexlopespereira/mba_enap/blob/main/CD/Notebooks/Aula7_PIB_GeoMunicipios_Bandeiras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL para carregar as bandeiras dos estados, PIB Percapita e Centróide dos municípios

In [1]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## 1) Centroides dos municipios

### 1.1) Download do arquivo de Localidades e limpeza/tratamento dos dados.
#### URL do arquivo exportado para xlsx https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx

In [2]:
df_xlsx = pd.read_excel('https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx', dtype={"CD_GEOCODM,C,20":str})
dfxlsx_geo = df_xlsx[['CD_GEOCODM,C,20','NM_CATEGOR,C,50','LONG,N,24,6','LAT,N,24,6']].rename(columns={'CD_GEOCODM,C,20':"cod_ibge", 'NM_CATEGOR,C,50':"categoria", 'LONG,N,24,6':"long", 'LAT,N,24,6':"lat"})
dfxlsx_geo['categoria'] = dfxlsx_geo['categoria'].str.strip()
dfxlsx_geo = dfxlsx_geo[dfxlsx_geo['categoria']=='CIDADE']
dfxlsx_geo['lat_long'] = dfxlsx_geo[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
dfxlsx_geo.head()

,cod_ibge,categoria,long,lat,lat_long
0,1100015,CIDADE,-61.999824,-11.935540,"-11.9355403047646,-61.9998238962936"
6,1100023,CIDADE,-63.033269,-9.908463,"-9.9084628665672,-63.0332692780484"
7,1100031,CIDADE,-60.544314,-13.499763,"-13.4997634596963,-60.5443135812009"
9,1100049,CIDADE,-61.442944,-11.433865,"-11.4338650286852,-61.4429442118224"
18,1100056,CIDADE,-60.818426,-13.195033,"-13.1950330320399,-60.8184261646815"


## 2) Join com a tabela de PIB per capita

### 2.1) Criar dataframe do PIB e População

In [3]:
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'hardy-messenger-229417' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df_pibpercapita = pd.io.gbq.read_gbq('''
SELECT pop.*, dsc.nome_municipio, pib.pib, pib.pib/pop.populacao as pibpercapita FROM `basedosdados.br_ibge_populacao.municipio` pop
LEFT JOIN `basedosdados.br_ibge_pib.municipio` pib on pop.id_municipio = pib.id_municipio and pib.ano = pop.ano
LEFT JOIN (
    select distinct (sc.id_municipio), sc.nome_municipio from `basedosdados.br_geobr_mapas.setor_censitario_2010` sc
    ) as dsc on dsc.id_municipio = pop.id_municipio
''', project_id=project_id)

df_pibpercapita.head()

/tmp/ipython-input-415532330.py:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_pibpercapita = pd.io.gbq.read_gbq('''


,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita
0,1991,RO,1100015,31981,Alta Floresta D'oeste,<NA>,NaN
1,1992,RO,1100015,34768,Alta Floresta D'oeste,<NA>,NaN
2,1993,RO,1100015,37036,Alta Floresta D'oeste,<NA>,NaN
3,1994,RO,1100015,39325,Alta Floresta D'oeste,<NA>,NaN
4,1995,RO,1100015,41574,Alta Floresta D'oeste,<NA>,NaN


### 2.1) Merge do dataframe do centroid com o dataframe do pib percapita

In [4]:
df_merge = df_pibpercapita.merge(dfxlsx_geo[['cod_ibge','lat_long']], how='left', left_on='id_municipio', right_on='cod_ibge')
del df_merge['cod_ibge']
df_merge.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita,lat_long
0,1991,RO,1100015,31981,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936"
1,1992,RO,1100015,34768,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936"
2,1993,RO,1100015,37036,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936"
3,1994,RO,1100015,39325,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936"
4,1995,RO,1100015,41574,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936"


## 3) Criar o dataframe das bandeiras

In [5]:
!wget https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/bandeiras/bandeiras.xlsx

--2025-08-24 00:56:10--  https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/bandeiras/bandeiras.xlsx
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexlopespereira/mba_enap/refs/heads/main/data/originais/bandeiras/bandeiras.xlsx [following]
--2025-08-24 00:56:10--  https://raw.githubusercontent.com/alexlopespereira/mba_enap/refs/heads/main/data/originais/bandeiras/bandeiras.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34558 (34K) [application/octet-stream]
Saving to: ‘bandeiras.xlsx’

bandeiras.xlsx      100%[===================>]  33.75K  --.-KB/s    in 0.

In [6]:
!ls

bandeiras.xlsx	sample_data


In [9]:
dfb = pd.read_excel("./bandeiras.xlsx")
dfb.head()

,ufNome,uf,Bandeira
0,Acre,AC,https://upload.wikimedia.org/wikipedia/commons...
1,Alagoas,AL,https://upload.wikimedia.org/wikipedia/commons...
2,Amapá,AP,https://upload.wikimedia.org/wikipedia/commons...
3,Amazonas,AM,https://upload.wikimedia.org/wikipedia/commons...
4,Bahia,BA,https://upload.wikimedia.org/wikipedia/commons...


In [10]:
dfb['Bandeira'][0]

'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Bandeira_do_Acre.svg/320px-Bandeira_do_Acre.svg.png'

### 3) Fazer o merge com a tabela do pib percapita

In [11]:
df_merge = df_merge.merge(dfb, left_on='sigla_uf', right_on='uf')

In [12]:
df_merge.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita,lat_long,ufNome,uf,Bandeira
0,1991,RO,1100015,31981,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936",Rondônia,RO,https://upload.wikimedia.org/wikipedia/commons...
1,1992,RO,1100015,34768,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936",Rondônia,RO,https://upload.wikimedia.org/wikipedia/commons...
2,1993,RO,1100015,37036,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936",Rondônia,RO,https://upload.wikimedia.org/wikipedia/commons...
3,1994,RO,1100015,39325,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936",Rondônia,RO,https://upload.wikimedia.org/wikipedia/commons...
4,1995,RO,1100015,41574,Alta Floresta D'oeste,<NA>,NaN,"-11.9355403047646,-61.9998238962936",Rondônia,RO,https://upload.wikimedia.org/wikipedia/commons...


### 4) Enviar o dataframe de resultado ao BigQuery.
#### Decida se lhe é conveniente reutilizar a tabela e o data source no Data Studio, ou se deseja criar uma nova tabela e um novo data source.
#### Dica 1: Se esta operação apenas adicionar colunas à tabela, sem remover outras, pode reutilizar a tabela e o data source.
#### Dica 2: Se reutilizar a tabela, atualize os campos do seu data source.


In [13]:
df_merge.to_gbq("enapdatasets.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

/tmp/ipython-input-1295702220.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_merge.to_gbq("enapdatasets.pibpercapita",
100%|██████████| 1/1 [00:00<00:00, 9686.61it/s]
